In [14]:
#Real one

In [17]:
import random
import math


# --- Classes and Setup ---
class Player:
    def __init__(self, name, matrix_position, attributes, is_goalkeeper=False):
        self.name = name
        self.matrix_position = matrix_position
        self.attributes = attributes
        self.is_goalkeeper = is_goalkeeper

    def get_attr(self, attr_name):
        return self.attributes.get(attr_name, 0)

class Team:
    def __init__(self, name, players):
        self.name = name
        self.players = players

OUTFIELD_ATTRS = [
    'Finishing', 'Tackling', 'Marking', 'Heading', 'Passing', 'Crossing', 'Ball Control',
    'Positioning', 'Vision', 'Composure', 'Work Rate',
    'Strength', 'Stamina', 'Acceleration', 'Agility', 'Jump Reach'
]
GOALKEEPER_ATTRS = [
    'Positioning', 'Command of Area', 'Composure', 'Work Rate',
    'Strength', 'Stamina', 'Acceleration', 'Agility', 'Jump Reach',
    'Reflexes', 'Handling', 'One-on-One', 'Aerial Reach'
]

# Hardcoded teams, as before
home_players = [
    Player("Jack Smith",      "GK", {attr: 10 for attr in GOALKEEPER_ATTRS}, is_goalkeeper=True),
    Player("Harry Brown",     "DL", {attr: 10 for attr in OUTFIELD_ATTRS}),
    Player("Oliver Jones",    "DC", {attr: 10 for attr in OUTFIELD_ATTRS}),
    Player("Liam Williams",   "DC", {attr: 10 for attr in OUTFIELD_ATTRS}),
    Player("Charlie Johnson", "DR", {attr: 10 for attr in OUTFIELD_ATTRS}),
    Player("George Miller",   "ML", {attr: 10 for attr in OUTFIELD_ATTRS}),
    Player("Noah Davis",      "MC", {attr: 10 for attr in OUTFIELD_ATTRS}),
    Player("Oscar Wilson",    "MC", {attr: 10 for attr in OUTFIELD_ATTRS}),
    Player("James Moore",     "MR", {attr: 10 for attr in OUTFIELD_ATTRS}),
    Player("Alfie Taylor",    "FC", {attr: 10 for attr in OUTFIELD_ATTRS}),
    Player("Thomas Anderson", "FC", {attr: 10 for attr in OUTFIELD_ATTRS}),
]

away_players = [
    Player("William Clark",   "GK", {attr: 5 for attr in GOALKEEPER_ATTRS}, is_goalkeeper=True),
    Player("Henry Hall",      "DC", {attr: 5 for attr in OUTFIELD_ATTRS}),
    Player("Jacob Lee",       "DC", {attr: 5 for attr in OUTFIELD_ATTRS}),
    Player("Leo Walker",      "DC", {attr: 5 for attr in OUTFIELD_ATTRS}),
    Player("Charlie White",   "DMR", {attr: 5 for attr in OUTFIELD_ATTRS}),
    Player("Freddie Harris",  "DML", {attr: 5 for attr in OUTFIELD_ATTRS}),
    Player("Archie Young",    "MC", {attr: 5 for attr in OUTFIELD_ATTRS}),
    Player("Ethan King",      "MC", {attr: 5 for attr in OUTFIELD_ATTRS}),
    Player("Alexander Wright","FC", {attr: 5 for attr in OUTFIELD_ATTRS}),
    Player("Joshua Scott",    "FC", {attr: 5 for attr in OUTFIELD_ATTRS}),
    Player("Logan Green",     "FC", {attr: 5 for attr in OUTFIELD_ATTRS}),
]

home_team = Team("Home United", home_players)
away_team = Team("Away FC", away_players)



In [25]:

#Creator matrices used for Creator phase of event
#------------------------------------------------
POSITIONS = ["DL", "DC", "DR", "DML", "DMC", "DMR",
             "ML", "MC", "MR", "OML", "OMC", "OMR", "FC"]

BASE_CREATOR_MATRIX = {
    "DL": 6, "DC": 1, "DR": 6,
    "DML": 10, "DMC": 6, "DMR": 10,
    "ML": 10, "MC": 10, "MR": 10,
    "OML": 12, "OMC": 15, "OMR": 12,
    "FC": 12
}


#Creator rows. columns are weighted chance for defender position
BASE_CREATOR_DEFEND_MATRIX = {
  "DL":  {"DL": 0,   "DC": 1,   "DR": 50,  "DML": 0,   "DMC": 30,  "DMR": 50,  "ML": 0,   "MC": 50,  "MR": 100, "OML": 0,   "OMC": 0,   "OMR": 100, "FC": 50},
    "DC":  {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 30,  "MC": 50,  "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 100},
    "DR":  {"DL": 50,  "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 100, "MC": 100, "MR": 0,   "OML": 50,  "OMC": 100, "OMR": 50,  "FC": 100},
    "DML": {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 0,   "MC": 50,  "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 0},
    "DMC": {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 100, "DMC": 50,  "DMR": 0,   "ML": 0,   "MC": 50,  "MR": 0,   "OML": 100, "OMC": 0,   "OMR": 0,   "FC": 0},
    "DMR": {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 50,  "DMR": 0,   "ML": 0,   "MC": 0,   "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 0},
    "ML":  {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 100, "MC": 50,  "MR": 100, "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 0},
    "MC":  {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 10,   "DMC": 50,  "DMR": 10,   "ML": 10,   "MC": 50,  "MR": 10,   "OML": 10,   "OMC": 10,   "OMR": 10,   "FC": 0},
    "MR":  {"DL": 100, "DC": 50,  "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 0,   "MC": 0,   "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 0},
    "OML": {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 0,   "MC": 0,   "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 0},
    "OMC": {"DL": 50,  "DC": 50,  "DR": 0,   "DML": 100, "DMC": 50,  "DMR": 0,   "ML": 0,   "MC": 0,   "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 30,  "FC": 0},
    "OMR": {"DL": 100, "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 0,   "MC": 0,   "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 0},
    "FC":  {"DL": 50,  "DC": 100, "DR": 50,  "DML": 10,  "DMC": 0,   "DMR": 10,  "ML": 0,   "MC": 0,   "MR": 0,   "OML": 10,  "OMC": 0,   "OMR": 0,   "FC": 0},
}

#Creator rows, chance/pass type on column
CREATOR_CHANCE_TYPE_MATRIX = {
    "DL":  {"Short": 15, "Long": 15, "Crossing": 55, "Through": 10, "Solo": 5},
    "DC":  {"Short": 20, "Long": 65, "Crossing": 5, "Through": 10, "Solo": 0},
    "DR":  {"Short": 15, "Long": 15, "Crossing": 55, "Through": 10, "Solo": 5},
    "DML": {"Short": 20, "Long": 15, "Crossing": 45, "Through": 15, "Solo": 5},
    "DMC": {"Short": 40, "Long": 35, "Crossing": 5, "Through": 15, "Solo": 5},
    "DMR": {"Short": 20, "Long": 15, "Crossing": 45, "Through": 15, "Solo": 5},
    "ML":  {"Short": 25, "Long": 5, "Crossing": 45, "Through": 10, "Solo": 15},
    "MC":  {"Short": 40, "Long": 20, "Crossing": 5, "Through": 25, "Solo": 10},
    "MR":  {"Short": 25, "Long": 5, "Crossing": 45, "Through": 10, "Solo": 15},
    "OML": {"Short": 15, "Long": 5, "Crossing": 40, "Through": 15, "Solo": 25},
    "OMC": {"Short": 40, "Long": 5, "Crossing": 5, "Through": 30, "Solo": 20},
    "OMR": {"Short": 15, "Long": 5, "Crossing": 40, "Through": 15, "Solo": 25},
    "FC":  {"Short": 35, "Long": 5, "Crossing": 5, "Through": 25, "Solo": 30},
}
CHANCE_TYPES = ["Short", "Long", "Crossing", "Through", "Solo"]


#Finsher matrices used for Finish phase of event
# if creator chance type = Short, long, cross, through, solo, select relevant matrix
#-------------------------------------------------------------------------------
# creator on row, short pass finisher on columns
BASE_FINISHER_SHORT_PASS_MATRIX = {
    "DL":  {"DL": 0, "DC": 30, "DR": 0, "DML": 0, "DMC": 50, "DMR": 0, "ML": 100, "MC": 100, "MR": 0, "OML": 100, "OMC": 50, "OMR": 0, "FC": 50},
    "DC":  {"DL": 50, "DC": 0, "DR": 50, "DML": 100, "DMC": 100, "DMR": 100, "ML": 100, "MC": 100, "MR": 100, "OML": 100, "OMC": 100, "OMR": 100, "FC": 50},
    "DR":  {"DL": 0, "DC": 30, "DR": 0, "DML": 0, "DMC": 50, "DMR": 0, "ML": 0, "MC": 100, "MR": 100, "OML": 0, "OMC": 50, "OMR": 100, "FC": 50},
    "DML": {"DL": 0, "DC": 30, "DR": 0, "DML": 0, "DMC": 50, "DMR": 0, "ML": 100, "MC": 100, "MR": 0, "OML": 100, "OMC": 50, "OMR": 0, "FC": 50},
    "DMC": {"DL": 0, "DC": 0, "DR": 0, "DML": 50, "DMC": 50, "DMR": 50, "ML": 100, "MC": 100, "MR": 100, "OML": 100, "OMC": 100, "OMR": 100, "FC": 50},
    "DMR": {"DL": 0, "DC": 30, "DR": 0, "DML": 0, "DMC": 50, "DMR": 0, "ML": 0, "MC": 100, "MR": 100, "OML": 0, "OMC": 50, "OMR": 100, "FC": 50},
    "ML":  {"DL": 30, "DC": 0, "DR": 0, "DML": 30, "DMC": 100, "DMR": 0, "ML": 0, "MC": 100, "MR": 0, "OML": 0, "OMC": 100, "OMR": 0, "FC": 50},
    "MC":  {"DL": 0, "DC": 0, "DR": 0, "DML": 30, "DMC": 50, "DMR": 30, "ML": 100, "MC": 100, "MR": 100, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "MR":  {"DL": 0, "DC": 0, "DR": 30, "DML": 0, "DMC": 100, "DMR": 30, "ML": 0, "MC": 100, "MR": 0, "OML": 0, "OMC": 100, "OMR": 0, "FC": 50},
    "OML": {"DL": 30, "DC": 0, "DR": 0, "DML": 30, "DMC": 30, "DMR": 0, "ML": 0, "MC": 50, "MR": 0, "OML": 0, "OMC": 100, "OMR": 0, "FC": 100},
    "OMC": {"DL": 0, "DC": 0, "DR": 0, "DML": 30, "DMC": 30, "DMR": 30, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "OMR": {"DL": 0, "DC": 0, "DR": 30, "DML": 0, "DMC": 30, "DMR": 30, "ML": 0, "MC": 50, "MR": 0, "OML": 0, "OMC": 100, "OMR": 0, "FC": 100},
    "FC":  {"DL": 0, "DC": 0, "DR": 0, "DML": 30, "DMC": 30, "DMR": 30, "ML": 30, "MC": 50, "MR": 30, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
}

# creator on row, crossing finisher on columns
BASE_FINISHER_CROSSING_MATRIX = {
    "DL":  {"DL": 0, "DC": 0, "DR": 30, "DML": 0, "DMC": 30, "DMR": 30, "ML": 0, "MC": 50, "MR": 50, "OML": 0, "OMC": 100, "OMR": 50, "FC": 100},
    "DC":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "DR":  {"DL": 30, "DC": 0, "DR": 0, "DML": 30, "DMC": 30, "DMR": 0, "ML": 50, "MC": 50, "MR": 0, "OML": 50, "OMC": 100, "OMR": 0, "FC": 100},
    "DML": {"DL": 0, "DC": 0, "DR": 30, "DML": 0, "DMC": 30, "DMR": 30, "ML": 0, "MC": 50, "MR": 50, "OML": 0, "OMC": 50, "OMR": 50, "FC": 100},
    "DMC": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "DMR": {"DL": 30, "DC": 0, "DR": 0, "DML": 30, "DMC": 30, "DMR": 0, "ML": 50, "MC": 50, "MR": 0, "OML": 50, "OMC": 50, "OMR": 0, "FC": 100},
    "ML":  {"DL": 0, "DC": 0, "DR": 30, "DML": 0, "DMC": 30, "DMR": 30, "ML": 0, "MC": 50, "MR": 50, "OML": 0, "OMC": 50, "OMR": 100, "FC": 100},
    "MC":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "MR":  {"DL": 30, "DC": 0, "DR": 0, "DML": 30, "DMC": 30, "DMR": 0, "ML": 50, "MC": 50, "MR": 0, "OML": 100, "OMC": 50, "OMR": 0, "FC": 100},
    "OML": {"DL": 0, "DC": 0, "DR": 30, "DML": 0, "DMC": 30, "DMR": 30, "ML": 0, "MC": 50, "MR": 50, "OML": 0, "OMC": 50, "OMR": 100, "FC": 100},
    "OMC": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "OMR": {"DL": 30, "DC": 0, "DR": 0, "DML": 30, "DMC": 30, "DMR": 0, "ML": 50, "MC": 50, "MR": 0, "OML": 100, "OMC": 50, "OMR": 0, "FC": 100},
    "FC":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
}

# creator on row, through ball finisher on columns
BASE_FINISHER_THROUGH_MATRIX = {
    "DL":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 0, "OML": 50, "OMC": 100, "OMR": 30, "FC": 100},
    "DC":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "DR":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 0, "MC": 50, "MR": 50, "OML": 30, "OMC": 100, "OMR": 50, "FC": 100},
    "DML": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 0, "OML": 100, "OMC": 100, "OMR": 50, "FC": 100},
    "DMC": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 30, "MC": 50, "MR": 30, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "DMR": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 0, "MC": 50, "MR": 50, "OML": 50, "OMC": 100, "OMR": 100, "FC": 100},
    "ML":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 0, "MC": 30, "MR": 0, "OML": 0, "OMC": 100, "OMR": 50, "FC": 100},
    "MC":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 30, "MC": 30, "MR": 30, "OML": 100, "OMC": 100, "OMR": 100, "FC": 1000},
    "MR":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 0, "MC": 30, "MR": 0, "OML": 50, "OMC": 100, "OMR": 0, "FC": 100},
    "OML": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 0, "MC": 50, "MR": 30, "OML": 0, "OMC": 100, "OMR": 50, "FC": 100},
    "OMC": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "OMR": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 30, "MC": 50, "MR": 0, "OML": 50, "OMC": 100, "OMR": 0, "FC": 100},
    "FC":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 30, "MC": 30, "MR": 30, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
}

# creator on row, long pass finisher on columns
BASE_FINISHER_LONG_PASS_MATRIX = {
    "DL":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "DC":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "DR":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "DML": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "DMC": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 30, "MC": 30, "MR": 30, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "DMR": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 50, "MR": 50, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "ML":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 0, "MC": 30, "MR": 30, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "MC":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 50, "MC": 0, "MR": 50, "OML": 100, "OMC": 50, "OMR": 100, "FC": 100},
    "MR":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 30, "MC": 30, "MR": 0, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "OML": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 30, "MC": 30, "MR": 30, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "OMC": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 30, "MC": 30, "MR": 30, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "OMR": {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 30, "MC": 30, "MR": 30, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
    "FC":  {"DL": 0, "DC": 0, "DR": 0, "DML": 0, "DMC": 0, "DMR": 0, "ML": 30, "MC": 30, "MR": 30, "OML": 100, "OMC": 100, "OMR": 100, "FC": 100},
}

#creator on row, solo finisher = creator, if goal random assist or no assist?
solo_drible_matrix = {pos: {pos: 1.0} for pos in home_norm.keys()}  # or whatever positions appear

#Finisher rows. columns are weighted chance for defender position
BASE_FINISH_DEFEND_MATRIX = {
    "DL":  {"DL": 0,   "DC": 1,   "DR": 50,  "DML": 0,   "DMC": 30,  "DMR": 50,  "ML": 0,   "MC": 50,  "MR": 100, "OML": 0,   "OMC": 0,   "OMR": 100, "FC": 50},
    "DC":  {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 30,  "MC": 50,  "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 100},
    "DR":  {"DL": 50,  "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 100, "MC": 100, "MR": 0,   "OML": 50,  "OMC": 100, "OMR": 50,  "FC": 100},
    "DML": {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 0,   "MC": 50,  "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 0},
    "DMC": {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 100, "DMC": 50,  "DMR": 0,   "ML": 0,   "MC": 50,  "MR": 0,   "OML": 100, "OMC": 0,   "OMR": 0,   "FC": 0},
    "DMR": {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 50,  "DMR": 0,   "ML": 0,   "MC": 0,   "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 0},
    "ML":  {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 100, "MC": 50,  "MR": 100, "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 0},
    "MC":  {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 10,   "DMC": 50,  "DMR": 10,   "ML": 10,   "MC": 50,  "MR": 10,   "OML": 10,   "OMC": 10,   "OMR": 10,   "FC": 0},
    "MR":  {"DL": 100, "DC": 50,  "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 0,   "MC": 0,   "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 0},
    "OML": {"DL": 0,   "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 0,   "MC": 0,   "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 0},
    "OMC": {"DL": 50,  "DC": 50,  "DR": 0,   "DML": 100, "DMC": 50,  "DMR": 0,   "ML": 0,   "MC": 0,   "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 30,  "FC": 0},
    "OMR": {"DL": 100, "DC": 1,   "DR": 0,   "DML": 0,   "DMC": 0,   "DMR": 0,   "ML": 0,   "MC": 0,   "MR": 0,   "OML": 0,   "OMC": 0,   "OMR": 0,   "FC": 0},
    "FC":  {"DL": 50,  "DC": 100, "DR": 50,  "DML": 10,  "DMC": 0,   "DMR": 10,  "ML": 0,   "MC": 0,   "MR": 0,   "OML": 10,  "OMC": 0,   "OMR": 0,   "FC": 0},
}


# Chance type on rows, type of finish on columns
CHANCE_TO_FINISH_TYPE_MATRIX = {
    "Crossing":{"FirstTime": 20, "Controlled": 20, "Header": 55, "Chip": 0,  "Finesse": 0,  "Power": 5},
    "Long":    {"FirstTime": 20, "Controlled": 20, "Header": 35, "Chip": 5,  "Finesse": 5,  "Power": 15},
    "Short":   {"FirstTime": 25, "Controlled": 30, "Header": 2,  "Chip": 3,  "Finesse": 10, "Power": 30},
    "Through": {"FirstTime": 25, "Controlled": 25, "Header": 10, "Chip": 15, "Finesse": 15, "Power": 10},
    "Solo":    {"FirstTime": 10, "Controlled": 25, "Header": 0,  "Chip": 10, "Finesse": 25, "Power": 30}
}


# EVAL FUNCTIONALITY
#------------------
# need to have all chance types, finish types and save eval. Plus any other eval.
EVENT_X_FORMULAS = {
    #----------chance types---------
    "Short": lambda attacker, defender:  
       # 0.5 * (attacker.get_attr("Passing") - defender.get_attr("Tackling"))+ 0.5 * (attacker.get_attr("Vision") - defender.get_attr("Positioning")),
        -100,
    "Crossing": lambda attacker, defender:       
         attacker.get_attr("Crossing") - defender.get_attr("Marking"),
        
    "Solo": lambda attacker, defender: 
        -100,
    "Through": lambda attacker, defender: 
        50,
    "Long": lambda attacker, defender: 
        -100,
    #----------Finish types---------
     "Short_finisher": lambda attacker, defender:  
       # 0.5 * (attacker.get_attr("Passing") - defender.get_attr("Tackling"))+ 0.5 * (attacker.get_attr("Vision") - defender.get_attr("Positioning")),
        -100,
    "Crossing_finisher": lambda attacker, defender:       
         attacker.get_attr("Crossing") - defender.get_attr("Marking"),
        
    "Solo_finisher": lambda attacker, defender: 
        -100,
    "Through_finisher": lambda attacker, defender: 
        50,
    "Long_finisher": lambda attacker, defender: 
        -100,
      #----------shot types---------
    "FirstTime": lambda attacker, defender: 
        100,
    "Controlled": lambda attacker, defender: 
        -100,
    "Header": lambda attacker, defender: 
        100,
    "Chip": lambda attacker, defender: 
        -100,
    "Finesse": lambda attacker, defender: 
        -100,
     "Power": lambda attacker, defender: 
        -100,
    #----------GK saves---------
    "FirstTime_save": lambda attacker, defender: 
        -100,
    "Controlled_save": lambda attacker, defender: 
        -100,
    "Header_save": lambda attacker, defender: 
        100,
    "Chip_save": lambda attacker, defender: 
        -100,
    "Finesse_save": lambda attacker, defender: 
        -100,
    "Power_save": lambda attacker, defender: 
        -100,
     #----------GK intercepts---------
    "Long_intercept": lambda attacker, defender: 
        -100,
    "Crossing_intercept": lambda attacker, defender: 
        -100,
    "Through_intercept": lambda attacker, defender: 
        100,
     #----------Set Pieces---------
    "Corner": lambda attacker, defender: 
        -100,
    "Freekick": lambda attacker, defender: 
        -100,
    "Penalty": lambda attacker, defender: 
        -100,
    #----------Counter---------
    "Counter": lambda attacker, defender: 
        -100,
    #----------GK Set piece saves -NOT NEEDED? ---------
    "Gk_Pen": lambda attacker, defender: 
        -100,
    "Gk_Free": lambda attacker, defender: 
        -100,
    "Gk_Corner": lambda attacker, defender: 
        -100,
    
}

# Sigmoid parameters, should probably be the same for all events
DEFAULT_PARAMS = {"a": 0.2, "c": 0, "L": 1}
EVENT_SIGMOID_PARAMS = {
    "Short":         DEFAULT_PARAMS,
    "Crossing":      DEFAULT_PARAMS,
    "Solo":          DEFAULT_PARAMS,
    "Through":       DEFAULT_PARAMS,
    "Long":          DEFAULT_PARAMS,
    "Short_finisher":         DEFAULT_PARAMS,
    "Crossing_finisher":      DEFAULT_PARAMS,
    "Solo_finisher":          DEFAULT_PARAMS,
    "Through_finisher":       DEFAULT_PARAMS,
    "Long_finisher":          DEFAULT_PARAMS,
    "FirstTime":     DEFAULT_PARAMS,
    "Controlled":    DEFAULT_PARAMS,
    "Header":        DEFAULT_PARAMS,
    "Chip":         DEFAULT_PARAMS,
    "Finesse":      DEFAULT_PARAMS,
    "Power":         DEFAULT_PARAMS,
     "FirstTime_save":     DEFAULT_PARAMS,
    "Controlled_save":    DEFAULT_PARAMS,
    "Header_save":        DEFAULT_PARAMS,
    "Chip_save":          DEFAULT_PARAMS,
    "Finesse_save":       DEFAULT_PARAMS,
    "Power_save":         DEFAULT_PARAMS,
    "Long_intercept":          DEFAULT_PARAMS,
    "Crossing_intercept":       DEFAULT_PARAMS,
    "Through_intercept":         DEFAULT_PARAMS,
     "Corner":         DEFAULT_PARAMS,
    "Penalty":         DEFAULT_PARAMS,
    "Freekick":         DEFAULT_PARAMS,
}

def sigmoid_eval(X, a, c, L):
    return c + L / (1 + math.exp(-a * X))

# Critical chance/failure to determine if counter attack or subsequent bonus happens
def crit_success_chance(x):
    # Only positive X, steep sigmoid or piecewise for easier tuning
    #return max(0.01, min(0.25, 0.10 + 0.02 * max(0, X-10)))  # e.g., 10% at X=10, +2% per X after
    return  x+0.5*(1-x)
    
def crit_fail_chance(x):
    #return max(0.01, min(0.25, 0.10 + 0.02 * max(0, -X-10)))    
    return x/2

def eval_event(event_type, attacker, defender):
    x_formula = EVENT_X_FORMULAS[event_type]
    params = EVENT_SIGMOID_PARAMS[event_type]
    X = x_formula(attacker, defender)
    prob = 1 - sigmoid_eval(X, params["a"], params["c"], params["L"])
    roll = random.random()
    crit_success = False
    crit_failure = False

   # Criticals logic
    cs_chance = 1- crit_success_chance(prob)
    cf_chance = crit_fail_chance(prob)
    
    if roll < cf_chance:
        crit_failure = True
    elif roll > cs_chance:
        crit_success = True
    
   # print( f"X: {X:.2f} | Sigmoid Prob: {prob:.2f} | Roll: {roll:.2f} | Crit_Success: {crit_success} (chance: {cs_chance:.2f}) | Crit_Failure: {crit_failure} (chance: {cf_chance:.2f})")

    return roll > prob, prob, X, crit_success, crit_failure

#use as:  success, prob, X, crit_success, crit_failure = eval_event("solo_dribble", creator, defender)


In [26]:
def get_formation_count(team):
    """Return a dict of position code -> number of players at that position for a team."""
    formation = {pos: 0 for pos in POSITIONS}
    for player in team.players:
        pos = player.matrix_position
        if pos in formation:
            formation[pos] += 1
    return formation

def build_match_creator_matrix(base_matrix, formation_count):
    """Return dict of position -> (raw, unnormalized score) for chance creation."""
    matrix = {}
    for pos in POSITIONS:
        matrix[pos] = base_matrix.get(pos, 0) * formation_count.get(pos, 0)
    return matrix

def normalize_matrix(matrix):
    total = sum(matrix.values())
    if total == 0:
        return {pos: 0 for pos in matrix}
    return {pos: value / total for pos, value in matrix.items()}

def get_team_match_creator_matrix(team, base_matrix=BASE_CREATOR_MATRIX):
    formation_count = get_formation_count(team)
    raw_matrix = build_match_creator_matrix(base_matrix, formation_count)
    norm_matrix = normalize_matrix(raw_matrix)
    return raw_matrix, norm_matrix

# Example usage:
home_raw, home_norm = get_team_match_creator_matrix(home_team)
away_raw, away_norm = get_team_match_creator_matrix(away_team)

print("Home Team Creator Matrix (raw):")
for pos, val in home_raw.items():
    if val > 0:
        print(f"{pos}: {val}")

print("\nHome Team Creator Matrix (normalized):")
for pos, prob in home_norm.items():
    if prob > 0:
        print(f"{pos}: {prob:.2%}")

print("\nAway Team Creator Matrix (raw):")
for pos, val in away_raw.items():
    if val > 0:
        print(f"{pos}: {val}")

print("\nAway Team Creator Matrix (normalized):")
for pos, prob in away_norm.items():
    if prob > 0:
        print(f"{pos}: {prob:.2%}")

Home Team Creator Matrix (raw):
DL: 6
DC: 2
DR: 6
ML: 10
MC: 20
MR: 10
FC: 24

Home Team Creator Matrix (normalized):
DL: 7.69%
DC: 2.56%
DR: 7.69%
ML: 12.82%
MC: 25.64%
MR: 12.82%
FC: 30.77%

Away Team Creator Matrix (raw):
DC: 3
DML: 10
DMR: 10
MC: 20
FC: 36

Away Team Creator Matrix (normalized):
DC: 3.80%
DML: 12.66%
DMR: 12.66%
MC: 25.32%
FC: 45.57%


In [27]:
def build_match_finisher_matrix_weighted(base_matrix, team):
    """
    For each creator (row) in base_matrix:
    - Only includes finishers in the team formation.
    - Multiplies each finisher's value by the number of players at that position.
    - Normalizes the row.
    """
    # Get player count per position
    position_count = {}
    for player in team.players:
        pos = player.matrix_position
        position_count[pos] = position_count.get(pos, 0) + 1

    match_matrix = {}
    for creator, finishers in base_matrix.items():
        if creator not in position_count:
            continue  # Skip creators not in formation
        # Multiply by number of finishers
        weighted = {}
        for finisher, base_val in finishers.items():
            if finisher in position_count:
                weighted[finisher] = base_val * position_count[finisher]
        total = sum(weighted.values())
        if total > 0:
            norm = {finisher: value / total for finisher, value in weighted.items()}
        else:
            norm = {finisher: 0 for finisher in weighted}
        match_matrix[creator] = norm
    return match_matrix


# Usage example for all four chance types:
home_short_pass_finishers = build_match_finisher_matrix_weighted(BASE_FINISHER_SHORT_PASS_MATRIX, home_team)
home_crossing_finishers   = build_match_finisher_matrix_weighted(BASE_FINISHER_CROSSING_MATRIX, home_team)
home_through_finishers    = build_match_finisher_matrix_weighted(BASE_FINISHER_THROUGH_MATRIX, home_team)
home_long_finishers       = build_match_finisher_matrix_weighted(BASE_FINISHER_LONG_PASS_MATRIX, home_team)

away_short_pass_finishers = build_match_finisher_matrix_weighted(BASE_FINISHER_SHORT_PASS_MATRIX, away_team)
away_crossing_finishers   = build_match_finisher_matrix_weighted(BASE_FINISHER_CROSSING_MATRIX, away_team)
away_through_finishers    = build_match_finisher_matrix_weighted(BASE_FINISHER_THROUGH_MATRIX, away_team)
away_long_finishers       = build_match_finisher_matrix_weighted(BASE_FINISHER_LONG_PASS_MATRIX, away_team)

# Example print
print("Home Short Pass Finisher Matrix (weighted, normalized):")
for creator, finishers in home_short_pass_finishers.items():
    print(f"{creator}:")
    for finisher, prob in finishers.items():
        print(f"  {finisher}: {prob:.2%}")
# Pretty print for Home Short Pass as an example:
print("Away Short Pass Finisher Matrix (normalized):")
for creator, finishers in away_short_pass_finishers.items():
    print(f"{creator}:")
    for finisher, prob in finishers.items():
        print(f"  {finisher}: {prob:.2%}")


# Example print
print("Home Cross Finisher Matrix (weighted, normalized):")
for creator, finishers in home_crossing_finishers.items():
    print(f"{creator}:")
    for finisher, prob in finishers.items():
        print(f"  {finisher}: {prob:.2%}")
# Pretty print for Home Short Pass as an example:
print("Away Cross Finisher Matrix (normalized):")
for creator, finishers in away_crossing_finishers.items():
    print(f"{creator}:")
    for finisher, prob in finishers.items():
        print(f"  {finisher}: {prob:.2%}")









Home Short Pass Finisher Matrix (weighted, normalized):
DL:
  DL: 0.00%
  DC: 13.04%
  DR: 0.00%
  ML: 21.74%
  MC: 43.48%
  MR: 0.00%
  FC: 21.74%
DC:
  DL: 8.33%
  DC: 0.00%
  DR: 8.33%
  ML: 16.67%
  MC: 33.33%
  MR: 16.67%
  FC: 16.67%
DR:
  DL: 0.00%
  DC: 13.04%
  DR: 0.00%
  ML: 0.00%
  MC: 43.48%
  MR: 21.74%
  FC: 21.74%
ML:
  DL: 9.09%
  DC: 0.00%
  DR: 0.00%
  ML: 0.00%
  MC: 60.61%
  MR: 0.00%
  FC: 30.30%
MC:
  DL: 0.00%
  DC: 0.00%
  DR: 0.00%
  ML: 16.67%
  MC: 33.33%
  MR: 16.67%
  FC: 33.33%
MR:
  DL: 0.00%
  DC: 0.00%
  DR: 9.09%
  ML: 0.00%
  MC: 60.61%
  MR: 0.00%
  FC: 30.30%
FC:
  DL: 0.00%
  DC: 0.00%
  DR: 0.00%
  ML: 8.33%
  MC: 27.78%
  MR: 8.33%
  FC: 55.56%
Away Short Pass Finisher Matrix (normalized):
DC:
  DC: 0.00%
  DML: 18.18%
  DMR: 18.18%
  MC: 36.36%
  FC: 27.27%
DML:
  DC: 20.45%
  DML: 0.00%
  DMR: 0.00%
  MC: 45.45%
  FC: 34.09%
DMR:
  DC: 20.45%
  DML: 0.00%
  DMR: 0.00%
  MC: 45.45%
  FC: 34.09%
MC:
  DC: 0.00%
  DML: 5.36%
  DMR: 5.36%
  MC: 35

In [28]:
def build_match_defender_matrix_weighted(base_matrix, attacking_team, defending_team):
    """
    For any base defender matrix (row: attacker/creator/finisher, col: defender),
    - Row keys: Only positions present in attacking team.
    - Col keys: Only positions present in defending team.
    - Each cell is base value * (num in attack pos) * (num in defend pos)
    - Each row is normalized to sum to 1 if not all zeros.
    """
    # Count how many players for each position in both teams
    attack_count = {}
    for p in attacking_team.players:
        k = p.matrix_position
        attack_count[k] = attack_count.get(k, 0) + 1
    defend_count = {}
    for p in defending_team.players:
        k = p.matrix_position
        defend_count[k] = defend_count.get(k, 0) + 1

    match_matrix = {}
    for row in base_matrix:
        if row not in attack_count:
            continue  # only use attacking positions in the team
        weighted_row = {}
        for col in base_matrix[row]:
            if col in defend_count:
                weighted_row[col] = base_matrix[row][col] * attack_count[row] * defend_count[col]
        total = sum(weighted_row.values())
        if total > 0:
            normalized_row = {col: v / total for col, v in weighted_row.items()}
        else:
            normalized_row = {col: 0 for col in weighted_row}
        match_matrix[row] = normalized_row
    return match_matrix


home_creator_vs_away_defend = build_match_defender_matrix_weighted(
    BASE_CREATOR_DEFEND_MATRIX,   # your base dict (creator->defender)
    attacking_team=home_team,
    defending_team=away_team
)

home_finish_vs_away_defend = build_match_defender_matrix_weighted(
    BASE_FINISH_DEFEND_MATRIX,   # your base dict (finisher->defender)
    attacking_team=away_team,
    defending_team=home_team
)

away_creator_vs_home_defend = build_match_defender_matrix_weighted(
    BASE_CREATOR_DEFEND_MATRIX,   # your base dict (creator->defender)
    attacking_team=home_team,
    defending_team=away_team
)

away_finish_vs_home_defend = build_match_defender_matrix_weighted(
    BASE_FINISH_DEFEND_MATRIX,   # your base dict (finisher->defender)
    attacking_team=away_team,
    defending_team=home_team
)


for attacker, defenders in home_creator_vs_away_defend.items():
    print(f"{attacker}:")
    for defender, prob in defenders.items():
        print(f"  {defender}: {prob:.2%}")

for attacker, defenders in away_finish_vs_home_defend.items():
    print(f"{attacker}:")
    for defender, prob in defenders.items():
        print(f"  {defender}: {prob:.2%}")

DL:
  DC: 0.99%
  DML: 0.00%
  DMR: 16.50%
  MC: 33.00%
  FC: 49.50%
DC:
  DC: 0.74%
  DML: 0.00%
  DMR: 0.00%
  MC: 24.81%
  FC: 74.44%
DR:
  DC: 0.60%
  DML: 0.00%
  DMR: 0.00%
  MC: 39.76%
  FC: 59.64%
ML:
  DC: 2.91%
  DML: 0.00%
  DMR: 0.00%
  MC: 97.09%
  FC: 0.00%
MC:
  DC: 2.44%
  DML: 8.13%
  DMR: 8.13%
  MC: 81.30%
  FC: 0.00%
MR:
  DC: 100.00%
  DML: 0.00%
  DMR: 0.00%
  MC: 0.00%
  FC: 0.00%
FC:
  DC: 93.75%
  DML: 3.12%
  DMR: 3.12%
  MC: 0.00%
  FC: 0.00%
DC:
  DL: 0.00%
  DC: 0.60%
  DR: 0.00%
  ML: 9.04%
  MC: 30.12%
  MR: 0.00%
  FC: 60.24%
DML:
  DL: 0.00%
  DC: 1.96%
  DR: 0.00%
  ML: 0.00%
  MC: 98.04%
  MR: 0.00%
  FC: 0.00%
DMR:
  DL: 0.00%
  DC: 100.00%
  DR: 0.00%
  ML: 0.00%
  MC: 0.00%
  MR: 0.00%
  FC: 0.00%
MC:
  DL: 0.00%
  DC: 1.64%
  DR: 0.00%
  ML: 8.20%
  MC: 81.97%
  MR: 8.20%
  FC: 0.00%
FC:
  DL: 16.67%
  DC: 66.67%
  DR: 16.67%
  ML: 0.00%
  MC: 0.00%
  MR: 0.00%
  FC: 0.00%


In [34]:
import random

def weighted_choice(items_with_probs):
    filtered = [(k, v) for k, v in items_with_probs.items() if v > 0]
    if not filtered:
        return None
    items, weights = zip(*filtered)
    return random.choices(items, weights=weights, k=1)[0]

def get_players_by_position(team, pos):
    return [p for p in team.players if p.matrix_position == pos]

def select_player_from_pos(team, pos, exclude=None):
    players = get_players_by_position(team, pos)
    if exclude is not None:
        players = [p for p in players if p != exclude]
    return random.choice(players) if players else None




class MatchSimulator:
    def __init__(self, home_team, away_team,
                 home_creator_matrix, away_creator_matrix,
                 home_chance_type_matrix, away_chance_type_matrix,
                 home_finisher_matrices, away_finisher_matrices,
                 home_creator_vs_away_defend, away_creator_vs_home_defend,
                 home_finish_vs_away_defend, away_finish_vs_home_defend,
                 chance_to_finish_matrix,
                 minutes=90):
        self.home_team = home_team
        self.away_team = away_team
        self.home_creator_matrix = home_creator_matrix
        self.away_creator_matrix = away_creator_matrix
        self.home_chance_type_matrix = home_chance_type_matrix
        self.away_chance_type_matrix = away_chance_type_matrix
        self.home_finisher_matrices = home_finisher_matrices
        self.away_finisher_matrices = away_finisher_matrices
        self.home_creator_vs_away_defend = home_creator_vs_away_defend
        self.away_creator_vs_home_defend = away_creator_vs_home_defend
        self.home_finish_vs_away_defend = home_finish_vs_away_defend
        self.away_finish_vs_home_defend = away_finish_vs_home_defend
        self.chance_to_finish_matrix = chance_to_finish_matrix
        self.minutes = minutes
        self.log = []

    def decide_event(self):
        return random.random() < 0.5  # 20% chance per minute

    def handle_penalty(self, attacking_team, defending_team, minute):
        goalkeeper = [p for p in defending_team.players if p.is_goalkeeper][0]
        penalty_taker = random.choice(attacking_team.players)  # could use role-specific logic
        success, prob, *_ = eval_event("Penalty", penalty_taker, goalkeeper)
        self.log.append((minute, "special_result", "penalty",
                         "goal" if success else "miss", f"{prob:.2f}",
                         penalty_taker.name, goalkeeper.name, attacking_team.name))

    def handle_freekick(self, attacking_team, defending_team, minute):
        free_kick_taker = random.choice(attacking_team.players)
        goalkeeper = [p for p in defending_team.players if p.is_goalkeeper][0]
        success, prob, *_ = eval_event("Freekick", free_kick_taker, goalkeeper)
        self.log.append((minute, "special_result", "free_kick",
                         "goal" if success else "saved", f"{prob:.2f}",
                         free_kick_taker.name, goalkeeper.name, attacking_team.name))
    
    def handle_corner(self, attacking_team, defending_team, minute):
        """
        Corner sequence:
          - Chance type = "Corner" (delivery eval).
          - Taker: team.corner_taker if set/in squad, else random.
          - Finisher: random from top-5 (Heading + Jump Reach) among attackers, excluding taker.
          - Finish defender: random from top-5 (Heading + Jump Reach) among defenders.
          - Finish type forced to "Header" (duel -> shot quality -> save).
        """
    
        # --- Select corner taker (creator)
        if hasattr(attacking_team, "corner_taker") and attacking_team.corner_taker in attacking_team.players:
            creator = attacking_team.corner_taker
        else:
            creator = random.choice(attacking_team.players)
        creator_pos = creator.matrix_position
    
        # --- Pick a creation defender (who contests the delivery)
        defend_matrix = self.home_creator_vs_away_defend if attacking_team == self.home_team else self.away_creator_vs_home_defend
        defend_probs = defend_matrix.get(creator_pos, {})
        if not defend_probs:
            present = {p.matrix_position for p in defending_team.players}
            defend_probs = {pos: 1/len(present) for pos in present}
        defender_pos = weighted_choice(defend_probs) or random.choice(list(defend_probs.keys()))
        defender = select_player_from_pos(defending_team, defender_pos) or random.choice(defending_team.players)
    
        # --- Evaluate the corner delivery (chance type = "Corner")
        chance_type = "Corner"
        success, prob, X, crit_s, crit_f = eval_event(chance_type, creator, defender)
        self.log.append((minute, "corner", "creation",
                         "success" if success else "fail",
                         f"{prob:.3f}", crit_s, crit_f,
                         creator.name, defender.name, chance_type, attacking_team.name))
        if not success:
            return
    
        # --- Build top-5 aerial candidates (Heading + Jump Reach)
        def top5_aerial(players, exclude=None):
            pool = [p for p in players if p is not exclude]
            pool.sort(key=lambda p: p.get_attr("Heading") + p.get_attr("Jump Reach"), reverse=True)
            return pool[:5] if len(pool) >= 5 else pool
    
        # Finisher: random from attackers' top-5, excluding taker
        top_attack = top5_aerial(attacking_team.players, exclude=creator)
        if not top_attack:
            return
        finisher = random.choice(top_attack)
        finisher_pos = finisher.matrix_position
    
        # Finish defender: random from defenders' top-5
        top_defend = top5_aerial(defending_team.players, exclude=None)
        if not top_defend:
            top_defend = defending_team.players[:]  # fallback
        finish_defender = random.choice(top_defend)
        finish_defender_pos = finish_defender.matrix_position
    
        # --- Duel for the header
        finish_type = "Header"
        success, prob, X, crit_s, crit_f = eval_event(f"{finish_type}_duel", finisher, finish_defender)
        self.log.append((minute, "corner", "finish",
                         "success" if success else "fail",
                         f"{prob:.3f}", crit_s, crit_f,
                         finisher.name, finish_defender.name, finish_type, attacking_team.name))
        if not success:
            return
    
        # --- Shot quality (Header)
        shot_on_target, shot_quality_prob, X_quality, crit_s_quality, crit_f_quality = eval_event(
            finish_type, finisher, finish_defender
        )
        self.log.append((minute, "corner", "shot_quality",
                         "on_target" if shot_on_target else "off_target",
                         f"{shot_quality_prob:.2f}",
                         finisher.name, finish_defender.name, finish_type, attacking_team.name))
    
        # --- Keeper save if on target
        if shot_on_target:
            gks = [p for p in defending_team.players if getattr(p, "is_goalkeeper", False)]
            goalkeeper = gks[0] if gks else random.choice(defending_team.players)
            saved, save_prob, X_save, crit_s_save, crit_f_save = eval_event(f"{finish_type}_save", finisher, goalkeeper)
            self.log.append((minute, "corner", "save",
                             "saved" if saved else "goal",
                             f"{save_prob:.2f}",
                             finisher.name, goalkeeper.name, finish_type, attacking_team.name))
        else:
            self.log.append((minute, "corner", "finish_outcome", "miss",
                             f"{shot_quality_prob:.2f}",
                             finisher.name, finish_defender.name, finish_type, attacking_team.name))
        
    def run(self):
        for minute in range(1, self.minutes + 1):
            if not self.decide_event():
                continue

            team = self.home_team if random.random() < 0.5 else self.away_team
            is_home = (team == self.home_team)
            opponent_team = self.away_team if is_home else self.home_team

            # --- CREATOR SELECTION ---
            creator_matrix = self.home_creator_matrix if is_home else self.away_creator_matrix
            creator_pos = weighted_choice(creator_matrix)
            if creator_pos is None:
                continue
            creator = select_player_from_pos(team, creator_pos)
            if creator is None:
                continue
            #self.log.append((minute, "attempt", "creator", creator.name, creator_pos, team.name))

            # --- CREATION DEFENDER SELECTION ---
            defend_matrix = self.home_creator_vs_away_defend if is_home else self.away_creator_vs_home_defend
            defend_probs = defend_matrix.get(creator_pos, {})
            defender_pos = weighted_choice(defend_probs)
            if defender_pos is None:
                # If no defender possible, pick random opponent
                possible_positions = set(p.matrix_position for p in opponent_team.players)
                defender_pos = random.choice(list(possible_positions))
            defender = select_player_from_pos(opponent_team, defender_pos)
            if defender is None:
                # Should be very rare!
                defender = random.choice(opponent_team.players)
            #self.log.append((minute, "attempt", "creation_defender", defender.name, defender_pos, opponent_team.name))

            # --- CHANCE TYPE SELECTION ---
            chance_type_matrix = self.home_chance_type_matrix if is_home else self.away_chance_type_matrix
            if creator_pos not in chance_type_matrix:
                continue
            chance_type = weighted_choice(chance_type_matrix[creator_pos])
            if chance_type is None:
                continue
            #self.log.append((minute, "attempt", "chance_type", chance_type, creator_pos, team.name))

            # --- EVALUATE CREATION ---
            success, prob, X, critical_success, critical_failure = eval_event(chance_type, creator, defender)
            creation_success = success
            self.log.append((minute, "result", "creation", "success" if creation_success else "fail",  f"{prob:.3f}",critical_success, critical_failure, creator.name, defender.name, chance_type, team.name))
             
            if creation_success:
                if random.random() < 0.73:  # 3% chance penalty in creation phase
                    self.log.append((minute, "special", "penalty_awarded", team.name, "during_creation"))
                    self.handle_penalty(team, opponent_team, minute)
                    continue
                elif random.random() < 0.05:  # 5% chance free kick in creation phase
                    self.log.append((minute, "special", "free_kick_awarded", team.name, "during_creation"))
                    self.handle_freekick(team, opponent_team, minute)
                    continue
                else:
                    continue  # Creation failed

            
            # --- FINISHER SELECTION ---
            finisher_matrix = self.home_finisher_matrices[chance_type] if is_home else self.away_finisher_matrices[chance_type]
            possible_finishers = dict(finisher_matrix[creator_pos])
            if chance_type == "Solo":
                finisher = creator
                finisher_pos = creator_pos
            else:
                # Remove creator's position if there are alternatives
                candidate_positions = [pos for pos in possible_finishers if pos != creator_pos]
                if not candidate_positions:  # Only possible is creator's own pos
                    candidate_positions = list(possible_finishers.keys())
                # Now select a valid position for finisher
                attempts = 0
                while True:
                    if not candidate_positions:
                        candidate_positions = list(possible_finishers.keys())
                    finisher_pos = weighted_choice({k: possible_finishers[k] for k in candidate_positions})
                    if finisher_pos is None:
                        finisher_pos = random.choice(list(possible_finishers.keys()))
                    finisher = select_player_from_pos(team, finisher_pos, exclude=creator)
                    if finisher is not None:
                        break
                    attempts += 1
                    if attempts > 10:
                        # As a fallback, pick any finisher (even creator)
                        finisher = creator
                        finisher_pos = creator_pos
                        break
            
             # 🔹 NEW: Chance of free kick or penalty during finishing
            if random.random() < 0.02:  # 2% penalty chance in finish phase
                self.log.append((minute, "special", "penalty_awarded", team.name, "during_finish"))
                self.handle_penalty(team, opponent_team, minute)
                continue
            elif random.random() < 0.04:  # 4% free kick chance in finish phase
                self.log.append((minute, "special", "free_kick_awarded", team.name, "during_finish"))
                self.handle_freekick(team, opponent_team, minute)
                continue
            
            #self.log.append((minute, "attempt", "finisher", finisher.name, finisher_pos, team.name))

           # --- DEFEND FINISH SELECTION ---
            fin_defend_matrix = self.home_finish_vs_away_defend if is_home else self.away_finish_vs_home_defend
            defend_probs_fin = dict(fin_defend_matrix.get(finisher_pos, {}))
            adjusted_probs = defend_probs_fin.copy()
            
            num_in_pos = len(get_players_by_position(opponent_team, defender_pos))
            if defender_pos in adjusted_probs and num_in_pos == 1:
                # Only one player at creation-defender position, exclude that position
                adjusted_probs[defender_pos] = 0
            
            # Renormalize
            total = sum(adjusted_probs.values())
            if total > 0:
                adjusted_probs = {k: v / total for k, v in adjusted_probs.items()}
            else:
                # fallback: pick any position present on the defending team
                all_positions = set(p.matrix_position for p in opponent_team.players)
                adjusted_probs = {p: 1/len(all_positions) for p in all_positions}
            
            # Try up to 10 times to avoid picking the same player as both defenders
            attempts = 0
            while True:
                finish_defender_pos = weighted_choice(adjusted_probs)
                if finish_defender_pos is None:
                    finish_defender_pos = random.choice(list(adjusted_probs.keys()))
                # Exclude the actual creation defender only if finish_defender_pos == defender_pos and num_in_pos == 1 (already handled above)
                finish_defender = select_player_from_pos(opponent_team, finish_defender_pos, exclude=defender if num_in_pos == 1 and finish_defender_pos == defender_pos else None)
                if finish_defender is not None:
                    break
                attempts += 1
                if attempts > 10:
                    # fallback: pick any opponent
                    finish_defender = random.choice(opponent_team.players)
                    finish_defender_pos = finish_defender.matrix_position
                    break
            
           # self.log.append((minute, "attempt", "finish_defender", finish_defender.name, finish_defender_pos, opponent_team.name))
            # --- FINISH TYPE SELECTION ---
            finish_type_probs = self.chance_to_finish_matrix[chance_type]
            finish_type = weighted_choice(finish_type_probs)
            if finish_type is None:
                finish_type = random.choice(list(finish_type_probs.keys()))
            #self.log.append((minute, "attempt", "finish_type", finish_type, chance_type, team.name))

            # --- GOALKEEPER INTERCEPTION CHECK ---
            if chance_type in ["Long", "Through", "Crossing"]:
                goalkeeper = [p for p in opponent_team.players if p.is_goalkeeper][0]
                intercepted, intercept_prob, X_intercept, crit_s_int, crit_f_int = eval_event(
                    f"{chance_type}_intercept",
                    finisher,
                    goalkeeper
                )
                self.log.append((
                    minute, "result", "goalkeeper_intercept",
                    "success" if intercepted else "fail",
                    f"{intercept_prob:.2f}",
                    finisher.name, goalkeeper.name,
                    chance_type, team.name
                ))
                if intercepted:
                    # Interception successful → skip finish phase
                    continue

            
         # --- EVALUATE FINISH ---
            success, prob, X, critical_success, critical_failure = eval_event(f"{chance_type}_finisher", finisher, finish_defender)
            finish_success = success
            self.log.append((minute, "result", "finish",
                             "success" if finish_success else "fail",
                             f"{prob:.3f}", critical_success, critical_failure,
                             finisher.name, finish_defender.name, finish_type, team.name))
            
            # --- EVALUATE SHOT QUALITY ---
            shot_on_target, shot_quality_prob, X_quality, crit_s_quality, crit_f_quality = eval_event(
                finish_type, finisher, finish_defender
            )
            
            self.log.append((
                minute, "result", "shot_quality",
                "on_target" if shot_on_target else "off_target",
                f"{shot_quality_prob:.2f}",
                finisher.name, finish_defender.name,
                finish_type, team.name
            ))
            
            # Step 2: Keeper save or miss outcome
            if shot_on_target:
                goalkeeper = [p for p in opponent_team.players if p.is_goalkeeper][0]
            
                saved, save_prob, X_save, crit_s_save, crit_f_save = eval_event(
                    f"{finish_type}_save", finisher, goalkeeper
                )
            
                self.log.append((
                    minute, "result", "save",
                    "saved" if saved else "goal",
                    f"{save_prob:.2f}",
                    finisher.name, goalkeeper.name,
                    finish_type, team.name
                ))
            
                # Corner only comes from saved shots
                if saved and random.random() < 0.3:
                    self.log.append((minute, "special", "corner_kick", team.name))
                    self.handle_corner(team, opponent_team, minute)
            
            else:
                # Off target outcome
                self.log.append((
                    minute, "result", "finish_outcome", "miss",
                    f"{shot_quality_prob:.2f}",
                    finisher.name, finish_defender.name,
                    finish_type, team.name
                ))
    
    def print_log(self):
        for entry in self.log:
            print("Min {:2}: {:7} {:16} {}".format(entry[0], entry[1], entry[2], entry[3:]))

# Example: 
sim = MatchSimulator(
     home_team, away_team,
     home_norm, away_norm,
     CREATOR_CHANCE_TYPE_MATRIX, CREATOR_CHANCE_TYPE_MATRIX,
     {
        "Short": home_short_pass_finishers,
        "Crossing": home_crossing_finishers,
        "Through": home_through_finishers,
        "Long": home_long_finishers,
        "Solo": solo_drible_matrix,  # if you have this
    },
    {
        "Short": away_short_pass_finishers,
        "Crossing": away_crossing_finishers,
        "Through": away_through_finishers,
        "Long": away_long_finishers,
        "Solo": solo_drible_matrix,  # if you have this
    },
     home_creator_vs_away_defend, away_creator_vs_home_defend,
     home_finish_vs_away_defend, away_finish_vs_home_defend,
     CHANCE_TO_FINISH_TYPE_MATRIX,
     minutes=90+random.randint(3,9)
 )
sim.run()
sim.print_log()

Min  2: result  creation         ('fail', '1.000', False, True, 'Ethan King', 'Jack Smith', 'Short', 'Away FC')
Min  2: result  finish           ('fail', '1.000', False, True, 'Charlie White', 'Liam Williams', 'Controlled', 'Away FC')
Min  2: result  shot_quality     ('off_target', '1.00', 'Charlie White', 'Liam Williams', 'Controlled', 'Away FC')
Min  2: result  finish_outcome   ('miss', '1.00', 'Charlie White', 'Liam Williams', 'Controlled', 'Away FC')
Min  5: result  creation         ('success', '0.269', True, False, 'Oscar Wilson', 'Freddie Harris', 'Crossing', 'Home United')
Min  5: special penalty_awarded  ('Home United', 'during_creation')
Min  5: special_result penalty          ('miss', '1.00', 'Jack Smith', 'William Clark', 'Home United')
Min  7: result  creation         ('success', '0.000', False, False, 'Joshua Scott', 'Liam Williams', 'Through', 'Away FC')
Min  7: special penalty_awarded  ('Away FC', 'during_creation')
Min  7: special_result penalty          ('miss', '1.00'

In [30]:
from collections import defaultdict, Counter

# ============ TEAM + PLAYER STATS STRUCTS ============

class OffDefSplit:
    """Holds per-type (Counter) and totals for a phase."""
    def __init__(self):
        self.attempt_by_type = Counter()
        self.success_by_type = Counter()  # offense: success means created/finished; defense: success means stopped it

    @property
    def attempts(self): return sum(self.attempt_by_type.values())
    @property
    def successes(self): return sum(self.success_by_type.values())


class ShootingSplit:
    """Shooting split by finish type."""
    def __init__(self):
        self.shots_by_type = Counter()
        self.shots_on_by_type = Counter()
        self.goals_by_type = Counter()

    @property
    def shots(self): return sum(self.shots_by_type.values())
    @property
    def shots_on(self): return sum(self.shots_on_by_type.values())
    @property
    def goals(self): return sum(self.goals_by_type.values())


class TeamStatsV2:
    def __init__(self, name):
        self.name = name
        # Creator and Finisher splits (offense & defense)
        self.creator_off = OffDefSplit()
        self.creator_def = OffDefSplit()
        self.finisher_off = OffDefSplit()
        self.finisher_def = OffDefSplit()
        # Shooting split
        self.shooting = ShootingSplit()

    def merge(self, other):
        self.creator_off.attempt_by_type.update(other.creator_off.attempt_by_type)
        self.creator_off.success_by_type.update(other.creator_off.success_by_type)
        self.creator_def.attempt_by_type.update(other.creator_def.attempt_by_type)
        self.creator_def.success_by_type.update(other.creator_def.success_by_type)

        self.finisher_off.attempt_by_type.update(other.finisher_off.attempt_by_type)
        self.finisher_off.success_by_type.update(other.finisher_off.success_by_type)
        self.finisher_def.attempt_by_type.update(other.finisher_def.attempt_by_type)
        self.finisher_def.success_by_type.update(other.finisher_def.success_by_type)

        self.shooting.shots_by_type.update(other.shooting.shots_by_type)
        self.shooting.shots_on_by_type.update(other.shooting.shots_on_by_type)
        self.shooting.goals_by_type.update(other.shooting.goals_by_type)


class PlayerStatsV2:
    def __init__(self, name, team):
        self.name = name
        self.team = team

        self.creator_off = OffDefSplit()
        self.creator_def = OffDefSplit()
        self.finisher_off = OffDefSplit()
        self.finisher_def = OffDefSplit()

        self.shooting = ShootingSplit()
        self.assists_by_chance_type = Counter()

    # convenience
    @property
    def goals(self): return self.shooting.goals
    @property
    def shots(self): return self.shooting.shots
    @property
    def shots_on(self): return self.shooting.shots_on
    @property
    def assists(self): return sum(self.assists_by_chance_type.values())

    def merge(self, other):
        self.creator_off.attempt_by_type.update(other.creator_off.attempt_by_type)
        self.creator_off.success_by_type.update(other.creator_off.success_by_type)
        self.creator_def.attempt_by_type.update(other.creator_def.attempt_by_type)
        self.creator_def.success_by_type.update(other.creator_def.success_by_type)

        self.finisher_off.attempt_by_type.update(other.finisher_off.attempt_by_type)
        self.finisher_off.success_by_type.update(other.finisher_off.success_by_type)
        self.finisher_def.attempt_by_type.update(other.finisher_def.attempt_by_type)
        self.finisher_def.success_by_type.update(other.finisher_def.success_by_type)

        self.shooting.shots_by_type.update(other.shooting.shots_by_type)
        self.shooting.shots_on_by_type.update(other.shooting.shots_on_by_type)
        self.shooting.goals_by_type.update(other.shooting.goals_by_type)

        self.assists_by_chance_type.update(other.assists_by_chance_type)


class MatchStatsV2:
    def __init__(self, home_team, away_team):
        self.team = {
            home_team.name: TeamStatsV2(home_team.name),
            away_team.name: TeamStatsV2(away_team.name),
        }
        self.player = {}  # (team, name) -> PlayerStatsV2

    def ps(self, team_name, player_name):
        key = (team_name, player_name)
        if key not in self.player:
            self.player[key] = PlayerStatsV2(player_name, team_name)
        return self.player[key]

    def merge(self, other):
        for t in self.team:
            self.team[t].merge(other.team[t])
        for key, pst in other.player.items():
            self.ps(*key).merge(pst)


# ============ LOG → STATS AGGREGATOR ============

def aggregate_match_log_to_stats_v2(sim) -> MatchStatsV2:
    ms = MatchStatsV2(sim.home_team, sim.away_team)
    home, away = sim.home_team.name, sim.away_team.name

    # Remember last successful creation per (minute, team) for assist credit
    last_creation = {}  # (minute, team) -> {"creator": str, "chance_type": str}

    for entry in sim.log:
        minute, section, tag = entry[:3]
        rest = entry[3:]

        # ------- CREATION PHASE -------
        # ("result","creation", outcome, prob, crit_s, crit_f, creator, defender, chance_type, team_name)
        if section == "result" and tag == "creation":
            outcome, prob, crit_s, crit_f, creator, defender, chance_type, atk_team = rest
            def_team = away if atk_team == home else home

            # Team offense & defense
            ms.team[atk_team].creator_off.attempt_by_type[chance_type] += 1
            ms.team[def_team].creator_def.attempt_by_type[chance_type] += 1

            # Player ledgers
            ms.ps(atk_team, creator).creator_off.attempt_by_type[chance_type] += 1
            ms.ps(def_team, defender).creator_def.attempt_by_type[chance_type] += 1

            if outcome == "success":
                # offense creation success
                ms.team[atk_team].creator_off.success_by_type[chance_type] += 1
                ms.ps(atk_team, creator).creator_off.success_by_type[chance_type] += 1
                # remember for assists
                last_creation[(minute, atk_team)] = {"creator": creator, "chance_type": chance_type}
            else:
                # defense creation success (they stopped it)
                ms.team[def_team].creator_def.success_by_type[chance_type] += 1
                ms.ps(def_team, defender).creator_def.success_by_type[chance_type] += 1

        # ------- FINISH DUEL -------
        # ("result","finish", outcome, prob, crit_s, crit_f, finisher, finish_defender, finish_type, team_name)
        elif section == "result" and tag == "finish":
            outcome, prob, crit_s, crit_f, finisher, fdef, finish_type, atk_team = rest
            def_team = away if atk_team == home else home

            # Team offense & defense
            ms.team[atk_team].finisher_off.attempt_by_type[finish_type] += 1
            ms.team[def_team].finisher_def.attempt_by_type[finish_type] += 1

            # Players
            ms.ps(atk_team, finisher).finisher_off.attempt_by_type[finish_type] += 1
            ms.ps(def_team, fdef).finisher_def.attempt_by_type[finish_type] += 1

            if outcome == "success":
                ms.team[atk_team].finisher_off.success_by_type[finish_type] += 1
                ms.ps(atk_team, finisher).finisher_off.success_by_type[finish_type] += 1
            else:
                # defense success (stopped finisher)
                ms.team[def_team].finisher_def.success_by_type[finish_type] += 1
                ms.ps(def_team, fdef).finisher_def.success_by_type[finish_type] += 1

        # ------- SHOT QUALITY -------
        # ("result","shot_quality", "on_target"/"off_target", prob, finisher, finish_defender, finish_type, team_name)
        elif section == "result" and tag == "shot_quality":
            outcome, prob, finisher, fdef, finish_type, atk_team = rest
            # attempts
            ms.team[atk_team].shooting.shots_by_type[finish_type] += 1
            ms.ps(atk_team, finisher).shooting.shots_by_type[finish_type] += 1
            # on/off
            if outcome == "on_target":
                ms.team[atk_team].shooting.shots_on_by_type[finish_type] += 1
                ms.ps(atk_team, finisher).shooting.shots_on_by_type[finish_type] += 1

        # ------- SAVE / GOAL RESOLUTION -------
        # ("result","save","saved"/"goal", prob, finisher, goalkeeper, finish_type, team_name)
        elif section == "result" and tag == "save":
            outcome, prob, finisher, goalkeeper, finish_type, atk_team = rest
            def_team = away if atk_team == home else home
            if outcome == "goal":
                # team & player goals by finish type
                ms.team[atk_team].shooting.goals_by_type[finish_type] += 1
                ms.ps(atk_team, finisher).shooting.goals_by_type[finish_type] += 1
                # assist if creator from same minute/team
                k = (minute, atk_team)
                if k in last_creation:
                    creator = last_creation[k]["creator"]
                    ch_type = last_creation[k]["chance_type"]
                    if creator != finisher:
                        ms.ps(atk_team, creator).assists_by_chance_type[ch_type] += 1

        # ------- CORNERS -------
        # ("special","corner_kick", team_name)
        elif section == "special" and tag == "corner_kick":
            # a corner is already handled as chance type "Corner" in your corner flow;
            # no extra shot lines here.

            # no-op; included here for completeness
            pass

        # Corner sub-events reuse same shapes as above using ("corner", phase, ...).
        # We already count those in the generic handlers (creation/finish/shot/save),
        # because your corner code logs with section "corner" not "result".
        elif section == "corner":
            phase = rest[0]
            if phase == "creation":
                outcome, prob, crit_s, crit_f, creator, defender, chance_type, atk_team = rest[1:]
                def_team = away if atk_team == home else home
                # treat identical to creation:
                ms.team[atk_team].creator_off.attempt_by_type[chance_type] += 1
                ms.team[def_team].creator_def.attempt_by_type[chance_type] += 1
                ms.ps(atk_team, creator).creator_off.attempt_by_type[chance_type] += 1
                ms.ps(def_team, defender).creator_def.attempt_by_type[chance_type] += 1
                if outcome == "success":
                    ms.team[atk_team].creator_off.success_by_type[chance_type] += 1
                    ms.ps(atk_team, creator).creator_off.success_by_type[chance_type] += 1
                    last_creation[(minute, atk_team)] = {"creator": creator, "chance_type": chance_type}
                else:
                    ms.team[def_team].creator_def.success_by_type[chance_type] += 1
                    ms.ps(def_team, defender).creator_def.success_by_type[chance_type] += 1

            elif phase == "finish":
                outcome, prob, finisher, fdef, finish_type, atk_team = rest[1:]
                def_team = away if atk_team == home else home
                ms.team[atk_team].finisher_off.attempt_by_type[finish_type] += 1
                ms.team[def_team].finisher_def.attempt_by_type[finish_type] += 1
                ms.ps(atk_team, finisher).finisher_off.attempt_by_type[finish_type] += 1
                ms.ps(def_team, fdef).finisher_def.attempt_by_type[finish_type] += 1
                if outcome == "success":
                    ms.team[atk_team].finisher_off.success_by_type[finish_type] += 1
                    ms.ps(atk_team, finisher).finisher_off.success_by_type[finish_type] += 1
                else:
                    ms.team[def_team].finisher_def.success_by_type[finish_type] += 1
                    ms.ps(def_team, fdef).finisher_def.success_by_type[finish_type] += 1

            elif phase == "shot_quality":
                outcome, prob, finisher, fdef, finish_type, atk_team = rest[1:]
                ms.team[atk_team].shooting.shots_by_type[finish_type] += 1
                if outcome == "on_target":
                    ms.team[atk_team].shooting.shots_on_by_type[finish_type] += 1
                ms.ps(atk_team, finisher).shooting.shots_by_type[finish_type] += 1
                if outcome == "on_target":
                    ms.ps(atk_team, finisher).shooting.shots_on_by_type[finish_type] += 1

            elif phase == "save":
                outcome, prob, finisher, goalkeeper, finish_type, atk_team = rest[1:]
                if outcome == "goal":
                    ms.team[atk_team].shooting.goals_by_type[finish_type] += 1
                    ms.ps(atk_team, finisher).shooting.goals_by_type[finish_type] += 1
                    k = (minute, atk_team)
                    if k in last_creation:
                        creator = last_creation[k]["creator"]
                        ch_type = last_creation[k]["chance_type"]
                        if creator != finisher:
                            ms.ps(atk_team, creator).assists_by_chance_type[ch_type] += 1

        # ------- PENALTIES / FREEKICKS (treated as shots on) -------
        elif section == "special_result" and tag == "penalty":
            outcome, prob, taker, goalkeeper, atk_team = rest
            finish_type = "Penalty"
            ms.team[atk_team].shooting.shots_by_type[finish_type] += 1
            ms.team[atk_team].shooting.shots_on_by_type[finish_type] += 1
            ms.ps(atk_team, taker).shooting.shots_by_type[finish_type] += 1
            ms.ps(atk_team, taker).shooting.shots_on_by_type[finish_type] += 1
            if outcome == "goal":
                ms.team[atk_team].shooting.goals_by_type[finish_type] += 1
                ms.ps(atk_team, taker).shooting.goals_by_type[finish_type] += 1

        elif section == "special_result" and tag == "free_kick":
            outcome, prob, taker, goalkeeper, atk_team = rest
            finish_type = "FreeKick"
            ms.team[atk_team].shooting.shots_by_type[finish_type] += 1
            ms.team[atk_team].shooting.shots_on_by_type[finish_type] += 1
            ms.ps(atk_team, taker).shooting.shots_by_type[finish_type] += 1
            ms.ps(atk_team, taker).shooting.shots_on_by_type[finish_type] += 1
            if outcome == "goal":
                ms.team[atk_team].shooting.goals_by_type[finish_type] += 1
                ms.ps(atk_team, taker).shooting.goals_by_type[finish_type] += 1

    return ms


# ============ RUNNERS ============

def run_match_and_collect_stats_v2(sim) -> MatchStatsV2:
    sim.run()
    return aggregate_match_log_to_stats_v2(sim)

def simulate_many_matches_v2(build_simulator_fn, n_matches=1000):
    agg = None
    per_match = []
    for _ in range(n_matches):
        sim = build_simulator_fn()
        mstats = run_match_and_collect_stats_v2(sim)
        per_match.append(mstats)
        if agg is None:
            agg = mstats
        else:
            agg.merge(mstats)
    return agg, per_match


# ============ SMALL PRINTERS (optional) ============

def print_team_stats_v2(ms: MatchStatsV2):
    for tname, ts in ms.team.items():
        print(f"\n=== Team: {tname} ===")
        print("Creator (off): attempts/success =", ts.creator_off.attempts, ts.creator_off.successes,
              "| by type:", dict(ts.creator_off.attempt_by_type), "/", dict(ts.creator_off.success_by_type))
        print("Creator (def): attempts/success =", ts.creator_def.attempts, ts.creator_def.successes,
              "| by type:", dict(ts.creator_def.attempt_by_type), "/", dict(ts.creator_def.success_by_type))
        print("Finisher (off): attempts/success =", ts.finisher_off.attempts, ts.finisher_off.successes,
              "| by type:", dict(ts.finisher_off.attempt_by_type), "/", dict(ts.finisher_off.success_by_type))
        print("Finisher (def): attempts/success =", ts.finisher_def.attempts, ts.finisher_def.successes,
              "| by type:", dict(ts.finisher_def.attempt_by_type), "/", dict(ts.finisher_def.success_by_type))
        print("Shots:", ts.shooting.shots, "On target:", ts.shooting.shots_on, "Goals:", ts.shooting.goals)
        print("Shots by type:", dict(ts.shooting.shots_by_type))
        print("On-target by type:", dict(ts.shooting.shots_on_by_type))
        print("Goals by type:", dict(ts.shooting.goals_by_type))

def print_player_stats_v2(ms: MatchStatsV2, team_name: str, top_k=10):
    rows = []
    for (t, name), ps in ms.player.items():
        if t != team_name: continue
        rows.append((
            ps.goals, ps.assists, ps.shots, name,
            dict(ps.creator_off.success_by_type),
            dict(ps.finisher_off.success_by_type)
        ))
    rows.sort(reverse=True)
    print(f"\nTop players ({team_name}) by G/A/Sh:")
    for g,a,s,name,cr_fin,fn_fin in rows[:top_k]:
        print(f"{name:20s}  G={g:2d} A={a:2d} Sh={s:2d} | Created={cr_fin} | Finished={fn_fin}")


In [33]:
# Build a fresh simulator each time (closure captures your teams & matrices)
def make_sim():
    return MatchSimulator(
        home_team, away_team,
        home_norm, away_norm,
        CREATOR_CHANCE_TYPE_MATRIX, CREATOR_CHANCE_TYPE_MATRIX,
        {
            "Short": home_short_pass_finishers,
            "Crossing": home_crossing_finishers,
            "Through": home_through_finishers,
            "Long": home_long_finishers,
            "Solo": solo_drible_matrix,
        },
        {
            "Short": away_short_pass_finishers,
            "Crossing": away_crossing_finishers,
            "Through": away_through_finishers,
            "Long": away_long_finishers,
            "Solo": solo_drible_matrix,
        },
        home_creator_vs_away_defend, away_creator_vs_home_defend,
        home_finish_vs_away_defend, away_finish_vs_home_defend,
        CHANCE_TO_FINISH_TYPE_MATRIX,
        minutes=90+random.randint(3,9)
    )

# One match:
sim = make_sim()
ms = run_match_and_collect_stats_v2(sim)
print_team_stats_v2(ms)
print_player_stats_v2(ms, home_team.name)
print_player_stats_v2(ms, away_team.name)

# 1000 matches aggregated:
agg, per = simulate_many_matches_v2(make_sim, n_matches=1000)
print("\n=== Aggregated over 1000 matches ===")
print_team_stats_v2(agg)
print_player_stats_v2(agg, home_team.name)
print_player_stats_v2(agg, away_team.name)


=== Team: Home United ===
Creator (off): attempts/success = 26 11 | by type: {'Short': 6, 'Through': 3, 'Crossing': 11, 'Long': 4, 'Solo': 2} / {'Through': 3, 'Crossing': 8}
Creator (def): attempts/success = 22 16 | by type: {'Short': 8, 'Through': 5, 'Crossing': 6, 'Long': 2, 'Solo': 1} / {'Short': 8, 'Crossing': 5, 'Long': 2, 'Solo': 1}
Finisher (off): attempts/success = 5 0 | by type: {'FirstTime': 2, 'Power': 2, 'Header': 1} / {}
Finisher (def): attempts/success = 6 5 | by type: {'Power': 2, 'Header': 1, 'FirstTime': 1, 'Finesse': 1, 'Controlled': 1} / {'Power': 2, 'Header': 1, 'FirstTime': 1, 'Finesse': 1}
Shots: 21 On target: 19 Goals: 2
Shots by type: {'FreeKick': 8, 'Penalty': 8, 'FirstTime': 2, 'Power': 2, 'Header': 1}
On-target by type: {'FreeKick': 8, 'Penalty': 8, 'FirstTime': 2, 'Header': 1}
Goals by type: {'FirstTime': 2}

=== Team: Away FC ===
Creator (off): attempts/success = 22 6 | by type: {'Short': 8, 'Through': 5, 'Crossing': 6, 'Long': 2, 'Solo': 1} / {'Through': 